In [1]:
import math
from typing import Sequence

import pandas as pd

# Подготовка данных <a name='preparing'></a>

Source: [book recommendation dataset](https://www.kaggle.com/datasets/arashnic/book-recommendation-dataset/data)

In [2]:
book = pd.read_csv('./Books.csv',low_memory=False)
rating = pd.read_csv('./Ratings.csv',low_memory=False)

df=book.merge(rating,how="left", on="ISBN")

df.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L,User-ID,Book-Rating
0,0195153448,Classical Mythology,Mark P. O. Morford,2002.0,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,2.0,0.0
1,0002005018,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,8.0,5.0
2,0002005018,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,11400.0,0.0
3,0002005018,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,11676.0,8.0
4,0060973129,Decision in Normandy,Carlo D'Este,1991.0,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,8.0,0.0


In [3]:
def fix_years(dataframe):
    dataframe['Year-Of-Publication'] = dataframe['Year-Of-Publication'].astype(str)
    
    for i in dataframe.index:
        book = dataframe.loc[i]
        if str(book['Year-Of-Publication']).endswith('.0'):
            book['Year-Of-Publication'] = book['Year-Of-Publication'][:-2]
        
        dataframe.loc[i] = book
            
    return dataframe

In [4]:
df = fix_years(df)

C:\Users\ipuzanov\AppData\Local\Temp\ipykernel_23472\861396345.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  book['Year-Of-Publication'] = book['Year-Of-Publication'][:-2]


In [5]:
df.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L,User-ID,Book-Rating
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,2.0,0.0
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,8.0,5.0
2,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,11400.0,0.0
3,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,11676.0,8.0
4,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,8.0,0.0


In [6]:
df.shape

(33090, 10)

In [7]:
df.info()
df.dropna(inplace=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33090 entries, 0 to 33089
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ISBN                 33090 non-null  object 
 1   Book-Title           33090 non-null  object 
 2   Book-Author          33090 non-null  object 
 3   Year-Of-Publication  33090 non-null  object 
 4   Publisher            33087 non-null  object 
 5   Image-URL-S          33087 non-null  object 
 6   Image-URL-M          33087 non-null  object 
 7   Image-URL-L          33087 non-null  object 
 8   User-ID              33079 non-null  float64
 9   Book-Rating          33079 non-null  float64
dtypes: float64(2), object(8)
memory usage: 3.8+ MB


In [8]:
df['User-ID'] = df['User-ID'].astype('int')
df["Book-Author"]=df["Book-Author"].astype("string")
df["Book-Author"]=df["Book-Author"].str.replace("DAN BROWN","Dan Brown")
df["Book-Author"]=df["Book-Author"].str.replace("Audrey Osofsky","Stephen King")
df["Book-Author"]=df["Book-Author"].str.replace("Dinah McCall","Stephen King")
df.drop(columns=["Image-URL-S","Image-URL-M"],inplace=True)

In [9]:
df=df[df["Book-Rating"]>0]
df["Book-Rating"].describe()

count    15131.000000
mean         7.682043
std          1.810948
min          1.000000
25%          7.000000
50%          8.000000
75%          9.000000
max         10.000000
Name: Book-Rating, dtype: float64

In [10]:
df["User-ID"].nunique(), df["Book-Title"].nunique()

(4473, 5763)

In [11]:
df.groupby('User-ID')['Book-Title'].agg('count').sort_values()

User-ID
7769        1
11648       1
11652       1
11654       1
26238       1
         ... 
23902      90
6575      101
7346      110
16795     201
11676    1323
Name: Book-Title, Length: 4473, dtype: int64

In [12]:
book_counts = pd.DataFrame(df["Book-Title"].value_counts())
book_counts.sort_values("Book-Title", ascending=False)

,Book-Title
The Lovely Bones: A Novel,78
Wild Animus,54
The Nanny Diaries: A Novel,53
The Da Vinci Code,51
The Red Tent (Bestselling Backlist),42
...,...
Domestic Bliss : Simple Ways to Add Style to Your Life,1
Grace by the Cup: A Break from the Daily Grind,1
The Colloghi Conspiracy,1
Shadowrun: Nosferatu (Shadowrun (Roc)),1


In [13]:
rare_book = book_counts[book_counts["Book-Title"] <= 20].index

In [14]:
common_book = df[~df["Book-Title"].isin(rare_book)]
common_book

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-L,User-ID,Book-Rating
32,0440234743,The Testament,John Grisham,1999,Dell,http://images.amazon.com/images/P/0440234743.0...,3329,8.0
36,0440234743,The Testament,John Grisham,1999,Dell,http://images.amazon.com/images/P/0440234743.0...,7346,9.0
37,0440234743,The Testament,John Grisham,1999,Dell,http://images.amazon.com/images/P/0440234743.0...,7352,8.0
38,0440234743,The Testament,John Grisham,1999,Dell,http://images.amazon.com/images/P/0440234743.0...,9419,5.0
41,0440234743,The Testament,John Grisham,1999,Dell,http://images.amazon.com/images/P/0440234743.0...,11224,6.0
...,...,...,...,...,...,...,...,...
32300,068419337X,SHIPPING NEWS,Annie Proulx,1993,Scribner,http://images.amazon.com/images/P/068419337X.0...,7371,7.0
32302,068419337X,SHIPPING NEWS,Annie Proulx,1993,Scribner,http://images.amazon.com/images/P/068419337X.0...,11676,3.0
32303,068419337X,SHIPPING NEWS,Annie Proulx,1993,Scribner,http://images.amazon.com/images/P/068419337X.0...,20388,8.0
33014,067087311X,Icy Sparks,Gwynn Rubio,1998,Viking Books,http://images.amazon.com/images/P/067087311X.0...,7841,10.0


In [15]:
user_book_df = common_book.pivot_table(index=["User-ID"], columns=["Book-Title"], values="Book-Rating")
user_book_df

Book-Title,A Bend in the Road,A Heartbreaking Work of Staggering Genius,A Painted House,Angels &amp; Demons,Bridget Jones's Diary,Confessions of a Shopaholic (Summer Display Opportunity),Divine Secrets of the Ya-Ya Sisterhood: A Novel,Dreamcatcher,Fahrenheit 451,Girl with a Pearl Earring,...,The Summons,The Testament,Timeline,To Kill a Mockingbird,"Tuesdays with Morrie: An Old Man, a Young Man, and Life's Greatest Lesson",Where the Heart Is (Oprah's Book Club (Paperback)),White Oleander : A Novel,White Oleander : A Novel (Oprah's Book Club),Wicked: The Life and Times of the Wicked Witch of the West,Wild Animus
User-ID,,,,,,,,,,,,,,,,,,,,,
26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN
91,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
114,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
165,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
242,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
278653,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
278663,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
278698,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Меры сходства

### Мера Жаккара

In [16]:
def get_intersection(a: Sequence, b: Sequence, default_value: int = 0) -> Sequence:
    return [(a[i], b[i]) for i in range(len(a)) if a[i] != default_value and b[i] != default_value]


def get_jaccard_measure(a: Sequence, b: Sequence, default_value: int = 0) -> float:
    intersection = len(get_intersection(a, b, default_value))
    # print(len(a), len(b), intersection, get_intersection(a, b))
    return intersection / (len(a) + len(b) - intersection)

### Косинусная мера

In [17]:
def get_cos_measure(a: Sequence, b: Sequence) -> float:    
    sum_a_square = sum_b_square = sum_vectors = 0
    for i in range(len(a)):
        sum_a_square += a[i] * a[i]
        sum_b_square += b[i] * b[i]
        sum_vectors += a[i] * b[i]
    
    return sum_vectors / (math.sqrt(sum_a_square) * math.sqrt(sum_b_square))

### Корреляция Пирсона

In [18]:
def get_pearson_measure(a: Sequence, b: Sequence) -> float:  
    n = len(a)
    sum_a_square = sum_b_square = sum_vectors = a_average = b_average = 0
    for i in range(n):
        sum_a_square += a[i] * a[i]
        sum_b_square += b[i] * b[i]
        sum_vectors += a[i] * b[i]
        a_average += a[i]
        b_average += b[i]
    
    a_average /= n
    b_average /= n
    
    return (sum_vectors - n * a_average * b_average) / (
        math.sqrt(sum_a_square - (n * a_average * a_average)) 
        * math.sqrt(sum_b_square - (n * b_average * b_average))
    )

# Исследование

In [19]:
user_book_df.head()

Book-Title,A Bend in the Road,A Heartbreaking Work of Staggering Genius,A Painted House,Angels &amp; Demons,Bridget Jones's Diary,Confessions of a Shopaholic (Summer Display Opportunity),Divine Secrets of the Ya-Ya Sisterhood: A Novel,Dreamcatcher,Fahrenheit 451,Girl with a Pearl Earring,...,The Summons,The Testament,Timeline,To Kill a Mockingbird,"Tuesdays with Morrie: An Old Man, a Young Man, and Life's Greatest Lesson",Where the Heart Is (Oprah's Book Club (Paperback)),White Oleander : A Novel,White Oleander : A Novel (Oprah's Book Club),Wicked: The Life and Times of the Wicked Witch of the West,Wild Animus
User-ID,,,,,,,,,,,,,,,,,,,,,
26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN
91,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
114,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
165,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
242,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
user_book_filled = user_book_df.fillna(0)
user_book_filled

Book-Title,A Bend in the Road,A Heartbreaking Work of Staggering Genius,A Painted House,Angels &amp; Demons,Bridget Jones's Diary,Confessions of a Shopaholic (Summer Display Opportunity),Divine Secrets of the Ya-Ya Sisterhood: A Novel,Dreamcatcher,Fahrenheit 451,Girl with a Pearl Earring,...,The Summons,The Testament,Timeline,To Kill a Mockingbird,"Tuesdays with Morrie: An Old Man, a Young Man, and Life's Greatest Lesson",Where the Heart Is (Oprah's Book Club (Paperback)),White Oleander : A Novel,White Oleander : A Novel (Oprah's Book Club),Wicked: The Life and Times of the Wicked Witch of the West,Wild Animus
User-ID,,,,,,,,,,,,,,,,,,,,,
26,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0
91,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
114,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
165,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
242,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
278653,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
278663,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
278698,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
book_user_df = user_book_df.transpose()
book_user_filled = book_user_df.fillna(0)
book_user_filled

User-ID,26,91,114,165,242,243,244,254,256,280,...,278543,278552,278554,278586,278633,278653,278663,278698,278798,278843
Book-Title,,,,,,,,,,,,,,,,,,,,,
A Bend in the Road,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Heartbreaking Work of Staggering Genius,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Painted House,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,8.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Angels &amp; Demons,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Bridget Jones's Diary,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,8.0,0.0,2.0,0.0,0.0,0.0,0.0,8.0
Confessions of a Shopaholic (Summer Display Opportunity),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Divine Secrets of the Ya-Ya Sisterhood: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Dreamcatcher,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Fahrenheit 451,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
def print_obj_self_measure(obj_scores):
    print(
        f'Jaccard:\t{get_jaccard_measure(obj_scores, obj_scores)}\n'
        f'Cos:\t\t{get_cos_measure(obj_scores, obj_scores)}\n'
        f'Pearson:\t{get_pearson_measure(obj_scores, obj_scores)}\n'
    )

In [23]:
def get_top_books(dataframe, top_measure):
    tmp_df = dataframe[dataframe.index.isin(top_measure.index)]
    return tmp_df.loc[:,tmp_df.notna().sum() > 2]

### По пользователю

In [24]:
for user_id in user_book_filled.index:
    uniq = user_book_filled.loc[user_id].unique()
    if len(uniq) > 5:
        print(user_id, uniq)

6251 [ 0.  4. 10.  6.  8.  7.]
6563 [ 0.  9.  8.  2. 10.  7.]
6575 [1. 7. 0. 4. 5. 8. 6. 2. 9.]
7125 [ 0.  4.  8.  7. 10.  6.]
8454 [ 0.  8.  7.  9.  6. 10.]
8801 [ 0. 10.  8.  6.  7.  9.]
11224 [0. 7. 9. 6. 5. 1.]
11676 [ 6.5         0.          8.          7.         10.          8.66666667
  9.          6.          1.          5.5         7.5         4.
  5.          8.5         9.5       ]
16795 [ 8.   0.   9.  10.   7.   5.   6.   4.   8.5]
17950 [ 0.  4. 10.  5.  9.  6.  8.]
21014 [0. 8. 9. 5. 7. 4.]
28177 [ 0. 10.  9.  5.  8.  6.]


In [25]:
user_id = 6563

In [26]:
user_scores = user_book_filled.loc[user_id].values

In [27]:
print_obj_self_measure(user_scores)

Jaccard:	0.10891089108910891
Cos:		0.9999999999999999
Pearson:	0.9999999999999998


In [28]:
user_book_filled['jaccard_measure'] = user_book_filled.apply(lambda raw: get_jaccard_measure(user_scores, raw.values), axis= 1)
user_book_filled['cos_measure'] = user_book_filled.apply(lambda raw: get_cos_measure(user_scores, raw.values), axis= 1)
user_book_filled['pearson_measure'] = user_book_filled.apply(lambda raw: get_pearson_measure(user_scores, raw.values), axis= 1)

In [29]:
user_book_filled.loc[user_id][['jaccard_measure', 'cos_measure', 'pearson_measure']]

Book-Title
jaccard_measure    0.108911
cos_measure        1.000000
pearson_measure    1.000000
Name: 6563, dtype: float64

In [30]:
top_jaccard = user_book_filled.sort_values("jaccard_measure", ascending=False)[['jaccard_measure']][:5]
top_jaccard

Book-Title,jaccard_measure
User-ID,
6563,0.108911
11676,0.087379
16795,0.056604
25216,0.027523
28177,0.027523


In [31]:
get_top_books(user_book_df, top_jaccard)

Book-Title,Good in Bed,House of Sand and Fog,Life of Pi,She's Come Undone (Oprah's Book Club),Summer Sisters,The Client,The Five People You Meet in Heaven,The Nanny Diaries: A Novel,The Reader,"Tuesdays with Morrie: An Old Man, a Young Man, and Life's Greatest Lesson",Wicked: The Life and Times of the Wicked Witch of the West
User-ID,,,,,,,,,,,
6563,9.0,NaN,NaN,2.0,9.0,9.0,10.0,7.0,NaN,10.0,8.0
11676,8.0,7.0,6.0,NaN,7.0,7.5,9.0,7.5,8.5,9.0,9.0
16795,9.0,9.0,10.0,10.0,9.0,5.0,8.0,9.0,5.0,NaN,NaN
25216,NaN,NaN,NaN,6.0,NaN,NaN,NaN,7.0,NaN,NaN,8.0
28177,NaN,10.0,9.0,NaN,NaN,NaN,NaN,8.0,6.0,9.0,8.0


In [32]:
top_cos = user_book_filled.sort_values("cos_measure", ascending=False)[['cos_measure']][:5]
top_cos

Book-Title,cos_measure
User-ID,
6563,1.000000
25996,0.464959
20703,0.456561
11676,0.451121
26057,0.439625


In [33]:
get_top_books(user_book_df, top_cos)

Book-Title,The Bean Trees,The Five People You Meet in Heaven,The Nanny Diaries: A Novel,"Tuesdays with Morrie: An Old Man, a Young Man, and Life's Greatest Lesson",Wicked: The Life and Times of the Wicked Witch of the West
User-ID,,,,,
6563,9.0,10.0,7.0,10.0,8.0
11676,NaN,9.0,7.5,9.0,9.0
20703,NaN,8.0,NaN,NaN,6.0
25996,9.0,6.0,NaN,NaN,NaN
26057,10.0,NaN,10.0,10.0,NaN


In [34]:
top_pearson = user_book_filled.sort_values("pearson_measure", ascending=False)[['pearson_measure']][:5]
top_pearson

Book-Title,pearson_measure
User-ID,
6563,1.000000
25996,0.434243
20703,0.424071
16009,0.400809
26057,0.361691


In [35]:
get_top_books(user_book_df, top_pearson)

Book-Title,The Bean Trees,The Five People You Meet in Heaven,Wicked: The Life and Times of the Wicked Witch of the West
User-ID,,,
6563,9.0,10.0,8.0
16009,NaN,6.0,9.0
20703,NaN,8.0,6.0
25996,9.0,6.0,NaN
26057,10.0,NaN,NaN


### По объекту

In [36]:
book_name="The Da Vinci Code"

In [37]:
book_scores = book_user_filled.loc[book_name].values

In [38]:
print_obj_self_measure(book_scores)

Jaccard:	0.026631853785900782
Cos:		0.9999999999999999
Pearson:	1.0000000000000002


In [39]:
book_user_filled['jaccard_measure'] = book_user_filled.apply(lambda col: get_jaccard_measure(book_scores, col.values), axis=1)
book_user_filled['cos_measure'] = book_user_filled.apply(lambda col: get_cos_measure(book_scores, col.values), axis=1)
book_user_filled['pearson_measure'] = book_user_filled.apply(lambda col: get_pearson_measure(book_scores, col.values), axis=1)

In [40]:
book_user_filled[['jaccard_measure', 'cos_measure', 'pearson_measure']].loc[book_name]

User-ID
jaccard_measure    0.026632
cos_measure        1.000000
pearson_measure    1.000000
Name: The Da Vinci Code, dtype: float64

In [41]:
book_jaccard = book_user_filled.sort_values("jaccard_measure", ascending=False)[['jaccard_measure']][:5]
book_jaccard

User-ID,jaccard_measure
Book-Title,
The Da Vinci Code,0.026632
The Lovely Bones: A Novel,0.005112
Good in Bed,0.003573
Angels &amp; Demons,0.003061
The Nanny Diaries: A Novel,0.002550


In [42]:
get_top_books(book_user_df, book_jaccard)

User-ID,5476,7283,10560,11224,11676,13273,15602,16795
Book-Title,,,,,,,,
Angels &amp; Demons,NaN,NaN,8.0,7.0,8.0,8.0,NaN,NaN
Good in Bed,NaN,8.0,8.0,NaN,8.0,NaN,10.0,9.0
The Da Vinci Code,10.0,9.0,10.0,9.0,9.0,9.0,9.0,10.0
The Lovely Bones: A Novel,4.0,NaN,8.0,9.0,5.0,8.0,9.0,10.0
The Nanny Diaries: A Novel,7.0,9.0,NaN,NaN,7.5,NaN,7.0,9.0


In [43]:
book_cos = book_user_filled.sort_values("cos_measure", ascending=False)[['cos_measure']][:5]
book_cos

User-ID,cos_measure
Book-Title,
The Da Vinci Code,1.000000
Good in Bed,0.207855
The Lovely Bones: A Novel,0.161445
Angels &amp; Demons,0.148543
The Reader,0.133176


In [44]:
get_top_books(book_user_df, book_cos)

User-ID,638,7283,10560,11224,11676,13273,15602,16795
Book-Title,,,,,,,,
Angels &amp; Demons,NaN,NaN,8.0,7.0,8.0,8.0,NaN,NaN
Good in Bed,NaN,8.0,8.0,NaN,8.0,NaN,10.0,9.0
The Da Vinci Code,10.0,9.0,10.0,9.0,9.0,9.0,9.0,10.0
The Lovely Bones: A Novel,10.0,NaN,8.0,9.0,5.0,8.0,9.0,10.0
The Reader,10.0,10.0,NaN,NaN,8.5,NaN,NaN,5.0


In [45]:
book_pearson = book_user_filled.sort_values("pearson_measure", ascending=False)[['pearson_measure']][:5]
book_pearson

User-ID,pearson_measure
Book-Title,
The Da Vinci Code,1.000000
Good in Bed,0.175925
Angels &amp; Demons,0.111290
The Lovely Bones: A Novel,0.106290
The Reader,0.102919


In [46]:
get_top_books(book_user_df, book_pearson)

User-ID,638,7283,10560,11224,11676,13273,15602,16795
Book-Title,,,,,,,,
Angels &amp; Demons,NaN,NaN,8.0,7.0,8.0,8.0,NaN,NaN
Good in Bed,NaN,8.0,8.0,NaN,8.0,NaN,10.0,9.0
The Da Vinci Code,10.0,9.0,10.0,9.0,9.0,9.0,9.0,10.0
The Lovely Bones: A Novel,10.0,NaN,8.0,9.0,5.0,8.0,9.0,10.0
The Reader,10.0,10.0,NaN,NaN,8.5,NaN,NaN,5.0
